# 2. ProcessPoolExecutor

Pula procesów przeważnie jest używana w zadaniach zależnych od CPU (CPU bounded tasks).

# Przykład: test pierwszości

Scenariusz: dla danej listy liczb sprawdź, które są liczbami pierwszymi.

Dane:
 - `NUMBERS`: lista testowanych liczb.
 - `is_prime`: funkcja, która sprawdza czy dana liczba jest liczbą pierwszą.
 - `timeit`: funkcja dekorująca, mierząca czas wykonania dekorowanej funkcji.

In [1]:
import math
import time


NUMBERS = [
    111111111111111,
    111111151111111,
    116666666666611,
    119315717513911,
    119755797430727,
    122334444555553,
    123123454321321,
    123456136101521,
    123456789101213,
    123456797654321,
    123467898764321,
    123571113171923,
]


def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True


def timeit(func):
    """Wraps the function for measuring its execution time."""
    
    def wrapped(*args, **kwargs):
        t_start = time.time()
        result = func(*args, **kwargs)
        print(f'Executed `{func.__name__}` in {(time.time() - t_start):.2f}s')
        return result
    
    return wrapped


### Przykładowy wynik dla liczby `88888888888889`,  wywołując funkcję  `is_prime`:

In [2]:
is_prime(88888888888889)

True

## Typowe zadanie sekwencyjne, które wywołuje funkcję dla każdej liczby


In [3]:
@timeit
def task_sequential(numbers):
    return [is_prime(n) for n in numbers]

#### Test `task_sequential`:


In [4]:
task_sequential(NUMBERS)

Executed `task_sequential` in 6.55s


[False, True, True, True, True, True, True, True, True, True, True, True]

## Zadanie

Proszę stworzyć funkcję `task_parallel`, która obliczy równolegle funkcję `is_prime` dla każdej liczby w liście `NUMBERS` dla poniższych scenariuszy:

 - używając metody `submit` z interfejsu `ProcessPoolExecutor`.
 - używając metody `map` z interfejsu `ProcessPoolExecutor`.


In [14]:
from concurrent.futures import ProcessPoolExecutor


@timeit
def task_parallel_map(numbers, n_workers=4):
    with ProcessPoolExecutor(n_workers) as pool:
        yield from pool.map(is_prime, numbers)

In [16]:
result = task_parallel_map(NUMBERS)
for _ in result:
    print(_)

Executed `task_parallel_map` in 0.00s
False
True
True
True
True
True
True
True
True
True
True
True


In [21]:
from concurrent.futures import ProcessPoolExecutor


@timeit
def task_parallel_submit(numbers, n_workers=4):
    with ProcessPoolExecutor(n_workers) as pool:
        futures = [pool.submit(is_prime, number) for number in numbers]
    return [future.result() for future in futures]

In [22]:
result = task_parallel_submit(NUMBERS)

Executed `task_parallel_submit` in 1.94s
